# LangChain: Models, Prompts and Output 

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [1]:
%env OPENAI_API_KEY=your-api-key


env: OPENAI_API_KEY=your-api-key


In [7]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = 'USE YOUR OWN API KEY'

In [8]:
import datetime
current_date = datetime.datetime.now().date()

# Defining the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [9]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [10]:
get_completion("Who is Albert Einstein ?")

'Albert Einstein was a German-born physicist who is widely regarded as one of the most influential scientists of the 20th century. He is best known for his theory of relativity and his famous equation E=mc², which describes the relationship between mass and energy. Einstein was also a pacifist and a civil rights activist, and he was awarded the Nobel Prize in Physics in 1921 for his work on theoretical physics.'

In [11]:
get_completion("Write the Bayesian Network Equation?")

'The Bayesian Network Equation is:\n\nP(X) = ∏ P(Xi | Parents(Xi))\n\nwhere P(X) is the joint probability distribution of all variables in the network, Xi is a variable in the network, and Parents(Xi) are the parents of Xi in the network. The equation states that the probability of each variable in the network is dependent on the probabilities of its parents. The joint probability distribution is calculated by multiplying the conditional probabilities of each variable given its parents.'

In [12]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [13]:
style = """American English \
in a calm and respectful tone
"""

In [14]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [15]:
response = get_completion(prompt)

In [16]:
response

"I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie. To add to my frustration, the warranty doesn't cover the cost of cleaning up my kitchen. Can you please help me out, friend?"

## Chat API : LangChain

Let's try how we can do the same using LangChain Model API like 'gpt-3.5-turbo-0301'.

In [34]:
#!pip install --upgrade langchain

### Model

In [12]:
from langchain.chat_models import ChatOpenAI

# Note that OpenAI charges to use the GPT API. (Free credits are sometimes provided to new users, but who gets credit and how long this deal will last is not transparent.) It costs $0.002 / 1000 tokens, where 1000 tokens equal about 750 words.

In [37]:
import openai
openai_api_key = 'your-actual-api-key'
openai.api_key = openai_api_key
chat = ChatOpenAI(openai_api_key=openai_api_key, model_name='gpt-3.5-turbo-0301', temperature=0.0)

In [26]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-0301', temperature=0.0, openai_api_key='your-api-key', openai_proxy='')

### Prompt template

In [27]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [28]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [29]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [30]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [31]:
customer_style = """American English \
in a calm and respectful tone
"""

In [32]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [33]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [34]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.messages.HumanMessage'>


In [35]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [ ]:
customer_response = chat(customer_messages)

In [80]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [81]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [82]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [ ]:
service_response = chat(service_messages)
print(service_response.content)

# END 